# Sequential neural network for wine gradings


In [2]:
import numpy as np
import pandas as pd
# sets the output device as qt
%matplotlib qt5 
import matplotlib.pyplot as plt
import xgboost
import tensorflow as tf
import shap

In [3]:
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, plot_roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split 
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier as knn
from sklearn.naive_bayes import GaussianNB as gnb
from sklearn import svm
from  sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.cross_decomposition import PLSRegression

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2

In [8]:
seed = 888
all_data = pd.read_excel('./2020_master_read_only.xlsx', sheet_name='2020_master').set_index('Vessel_Out')
X = all_data.iloc[:,13:78]
y = all_data['Grade']
X.dropna(axis=0, how='any', inplace=True)
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.15, random_state=seed, stratify=y, shuffle=True)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cat_y = to_categorical(y, num_classes=3)
cat_y_train = to_categorical(y_train, num_classes=3)
cat_y_test = to_categorical(y_test, num_classes=3)

In [9]:
dl_model = Sequential()
dl_model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_dim=X.shape[1]))
dl_model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
dl_model.add(Dense(3, activation='softmax'))
adamoptimise = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-01-23 23:46:57.543624: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-23 23:46:57.544663: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
The `lr` argument is deprecated, use `learning_rate` instead.


In [10]:
dl_model.compile(loss='categorical_crossentropy', optimizer=adamoptimise, metrics=['accuracy'])
dl_history = dl_model.fit(X_train.values, cat_y_train, epochs=500, batch_size=32)

2022-01-23 23:47:47.272365: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-01-23 23:47:47.426251: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/500
8/8 [==============================] - 0s 9ms/step - loss: 2473.0308 - accuracy: 0.5185
Epoch 2/500
8/8 [==============================] - 0s 7ms/step - loss: 1773.3922 - accuracy: 0.5144
Epoch 3/500
8/8 [==============================] - 0s 7ms/step - loss: 1162.9320 - accuracy: 0.4156
Epoch 4/500
8/8 [==============================] - 0s 7ms/step - loss: 864.3894 - accuracy: 0.3909
Epoch 5/500
8/8 [==============================] - 0s 7ms/step - loss: 508.4913 - accuracy: 0.4568
Epoch 6/500
8/8 [==============================] - 0s 7ms/step - loss: 245.5281 - accuracy: 0.3786
Epoch 7/500
8/8 [==============================] - 0s 7ms/step - loss: 243.6152 - accuracy: 0.2922
Epoch 8/500
8/8 [==============================] - 0s 7ms/step - loss: 204.4390 - accuracy: 0.3786
Epoch 9/500
8/8 [==============================] - 0s 7ms/step - loss: 184.5706 - accuracy: 0.3704
Epoch 10/500
8/8 [==============================] - 0s 11ms/step - loss: 166.7851 - accuracy: 0.4280
Epoch

In [11]:
plt.plot(dl_history.history['accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

In [12]:
plt.plot(dl_history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper right')
plt.show()